This notebook shows how to:

*   Fine-tune TinyLlama with QLoRa
*   Quantize TinyLlama with GPTQ and AWQ
*   Do inference with TinyLlama
*   Benchmark TinyLlama with optimum-benchmark and the Evaluation Harness.

Each section of this notebook can be run independently.



# Inference

In [ ]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(1234)  # For reproducibility

prompt = "The best recipe for pasta is"

checkpoint = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.float16, device_map="cuda")

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, num_beams=5, do_sample=True, max_new_tokens=150)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(result)

The best recipe for pasta is the one you make at home, and this one is a great example of that. This recipe is so easy to make, and it's so delicious that you'll want to make it over and over again. It's also a great way to use up leftover pasta sauce.
1. In a large pot, heat the olive oil over medium-high heat. Add the onion and garlic and cook, stirring occasionally, until the onion is soft and translucent, about 5 minutes.
2. Add the zucchini and cook, stirring occasionally, until the zucchini is soft and translucent, about 5 minutes.
3


# Quantization

GPTQ

More details about the GPTQ quantization in this article:

[Quantize and Fine-tune LLMs with GPTQ Using Transformers and TRL](https://kaitchup.substack.com/p/quantize-and-fine-tune-llms-with)


In [ ]:
!pip install --upgrade transformers auto-gptq accelerate datasets
!python -m pip install git+https://github.com/huggingface/optimum.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transfo

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer
import torch
model_path = 'TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T'
w = 4 #quantization to 4-bit. Change to 2, 3, or 8 to quantize with another precision

quant_path = 'TinyLlama-1.1B-intermediate-step-1431k-3T-gptq-'+str(w)+'bit'

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
quantizer = GPTQQuantizer(bits=w, dataset="c4-new", model_seqlen = 2048)
quantized_model = quantizer.quantize_model(model, tokenizer)

quantized_model.save_pretrained("./"+quant_path, safetensors=True)
tokenizer.save_pretrained("./"+quant_path)

AWQ

More details about the AWQ quantization in this article:

[Fast and Small Llama 2 with Activation-Aware Quantization (AWQ)
](https://kaitchup.substack.com/p/fast-and-small-llama-2-with-activation)


In [ ]:
!pip install --upgrade transformers autoawq optimum accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.0/234.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T'
quant_path = 'TinyLlama-1.1B-intermediate-step-1431k-3T-awq-4bit'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model and tokenizer
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model with safetensors
model.save_quantized("./"+quant_path, safetensors=True)
tokenizer.save_pretrained("./"+quant_path)


config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating validation split: 0 examples [00:00, ? examples/s]

AWQ: 100%|██████████| 22/22 [04:48<00:00, 13.09s/it]


('./TinyLlama-1.1B-intermediate-step-1431k-3T-awq-4bit/tokenizer_config.json',
 './TinyLlama-1.1B-intermediate-step-1431k-3T-awq-4bit/special_tokens_map.json',
 './TinyLlama-1.1B-intermediate-step-1431k-3T-awq-4bit/tokenizer.model',
 './TinyLlama-1.1B-intermediate-step-1431k-3T-awq-4bit/added_tokens.json',
 './TinyLlama-1.1B-intermediate-step-1431k-3T-awq-4bit/tokenizer.json')

# Fine-tuning

QLoRA

More details about QLoRA fine-tuning in this article:

[QLoRa: Fine-Tune a Large Language Model on Your GPU](https://kaitchup.substack.com/p/qlora-fine-tune-a-large-language-model-on-your-gpu-27bed5a03e2b)

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer

model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

ds = load_dataset("timdettmers/openassistant-guanaco")
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Gradient checkpointing is used by default but not compatible with caching

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

training_arguments = TrainingArguments(
        output_dir="./results_qlora",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=50,
        logging_steps=50,
        learning_rate=2e-5,
        eval_steps=50,
        max_steps=300,
        warmup_steps=30,
        lr_scheduler_type="linear",
)

trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        eval_dataset=ds['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when t

Step,Training Loss,Validation Loss
50,1.836900,1.810579
100,1.700600,1.707797
150,1.666100,1.674216
200,1.613400,1.663975
250,1.641300,1.658830
300,1.621400,1.657307


***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
Saving model checkpoint to ./results_qlora/tmp-checkpoint-50
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-intermediate-step-1431k-3T/snapshots/036fa4651240b9a1487f709833b9e4b96b4c1574/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.37.2",
  "use_cache": true,
  "vocab_size": 32000
}

tok

TrainOutput(global_step=300, training_loss=1.6799505106608073, metrics={'train_runtime': 1088.1487, 'train_samples_per_second': 2.206, 'train_steps_per_second': 0.276, 'total_flos': 7676537722306560.0, 'train_loss': 1.6799505106608073, 'epoch': 0.24})

LoRA

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer

model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

ds = load_dataset("timdettmers/openassistant-guanaco")

model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}
)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Gradient checkpointing is used by default but not compatible with caching

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

training_arguments = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=50,
        logging_steps=50,
        learning_rate=2e-5,
        eval_steps=50,
        max_steps=300,
        warmup_steps=30,
        lr_scheduler_type="linear",
)
model.gradient_checkpointing_enable()
trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        eval_dataset=ds['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when t

Step,Training Loss,Validation Loss
50,1.791100,1.783625
100,1.660600,1.667348
150,1.631700,1.644055
200,1.583200,1.632580
250,1.609000,1.627794
300,1.589200,1.626145


***** Running Evaluation *****
  Num examples = 518
  Batch size = 8
Saving model checkpoint to ./results/tmp-checkpoint-50
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-intermediate-step-1431k-3T/snapshots/036fa4651240b9a1487f709833b9e4b96b4c1574/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.37.2",
  "use_cache": true,
  "vocab_size": 32000
}

tokenizer

TrainOutput(global_step=300, training_loss=1.6441412607828776, metrics={'train_runtime': 1059.4688, 'train_samples_per_second': 2.265, 'train_steps_per_second': 0.283, 'total_flos': 7676537722306560.0, 'train_loss': 1.6441412607828776, 'epoch': 0.24})

# Benchmarking

With optimum-benchmark: Memory-Efficiency and Inference Speed

More details about using optimum-benchmark in this article:

[Optimum-Benchmark: How Fast and Memory-Efficient Is Your LLM?](https://kaitchup.substack.com/p/optimum-benchmark-how-fast-and-memory)

In [ ]:
!python -m pip install git+https://github.com/huggingface/optimum-benchmark.git
!pip install bitsandbytes
!pip install auto-gptq
!pip install autoawq
!pip install --upgrade transformers

  Cloning https://github.com/huggingface/optimum-benchmark.git to /tmp/pip-req-build-zos1a3ua
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/optimum-benchmark.git /tmp/pip-req-build-zos1a3ua
  Resolved https://github.com/huggingface/optimum-benchmark.git to commit 777070b4267dfdebc89aa62812b1f20e75a1eebd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/RadeonOpenCompute/pyrsmi.git to /tmp/pip-install-dvnndbdh/pyrsmi_7da4e24235cc4cb39b89828cdf38846e
  Running command git clone --filter=blob:none --quiet https://github.com/RadeonOpenCompute/pyrsmi.git /tmp/pip-install-dvnndbdh/pyrsmi_7da4e24235cc4cb39b89828cdf38846e
  Resolved https://github.com/RadeonOpenCompute/pyrsmi.git to commit 931720d1b120cf3120beea578af632f2f40fd9ff
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 3.3 MB/s eta

Benchmarking TinyLlama fp16

In [ ]:
import os
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: inference # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID

experiment_name: tinyllama-3T-fp16
model: TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  torch_dtype: float16 #The model will be loaded with fp16

benchmark:
  memory: true #We will monitor the memory usage
  warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

  new_tokens: 1000 #Inference will generate 1000 tokens
  input_shapes:
    sequence_length: 512 #Prompt will have 512 tokens
    batch_size: 4
"""

with open("tinyllama-fp16.yaml", 'w') as f:
  f.write(YAML_DEFAULT)
os.system("optimum-benchmark --config-dir ./ --config-name tinyllama-fp16")

0

Benchmarking TinyLlama 4-bit GPTQ

In [ ]:
import os
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: inference # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID

experiment_name: tinyllama-3T-gptq-4bit
model: kaitchup/TinyLlama-1.1B-intermediate-step-1431k-3T-gptq-4bit #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  torch_dtype: float16 #The model will be loaded with fp16

benchmark:
  memory: true #We will monitor the memory usage
  warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

  new_tokens: 1000 #Inference will generate 1000 tokens
  input_shapes:
    sequence_length: 512 #Prompt will have 512 tokens
    batch_size: 4
"""

with open("tinyllama-gptq-4bit.yaml", 'w') as f:
  f.write(YAML_DEFAULT)
os.system("optimum-benchmark --config-dir ./ --config-name tinyllama-gptq-4bit")

0

Benchmarking TinyLlama 4-bit AWQ

In [ ]:
import os
YAML_DEFAULT="""
defaults:
  - backend: pytorch # default backend
  - benchmark: inference # we will monitor the inference
  - launcher: process
  - experiment # inheriting from experiment config
  - _self_ # for hydra 1.1 compatibility
  - override hydra/job_logging: colorlog # colorful logging
  - override hydra/hydra_logging: colorlog # colorful logging

hydra:
  run:
    dir: experiments/${experiment_name} #The results will be reported in this directory. Note that "experiment_name" refers to the configuration field name "experiment_name" below
  sweep:
    dir: experiments/${experiment_name}
  job:
    chdir: true
    env_set: #These are environment variable that you may want to set before running the benchmark
      CUDA_VISIBLE_DEVICES: 0
      CUDA_DEVICE_ORDER: PCI_BUS_ID

experiment_name: tinyllama-3T-awq-4bit
model: kaitchup/TinyLlama-1.1B-intermediate-step-1431k-3T-awq-4bit #The model that we want to evaluate. It can be from the Hugging Face Hub or local directory
device: cuda #Which device to use for the benchmark. We will use CUDA, i.e., the GPU

backend:
  torch_dtype: float16 #The model will be loaded with fp16

benchmark:
  memory: true #We will monitor the memory usage
  warmup_runs: 10 #Before the monitoring starts, the inference will be run 10 times for warming up

  new_tokens: 1000 #Inference will generate 1000 tokens
  input_shapes:
    sequence_length: 512 #Prompt will have 512 tokens
    batch_size: 4
"""

with open("tinyllama-awq-4bit.yaml", 'w') as f:
  f.write(YAML_DEFAULT)
os.system("optimum-benchmark --config-dir ./ --config-name tinyllama-awq-4bit")

0

With Evaluation Harness


More details about using the Evaluation Harness in this article:

[Behind the OpenLLM Leaderboard: The Evaluation Harness](https://kaitchup.substack.com/p/behind-the-openllm-leaderboard-the)

In [ ]:
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git
!pip install bitsandbytes
!pip install --upgrade transformers
!pip install auto-gptq optimum autoawq

  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git to /tmp/pip-req-build-4hqgpyxn
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness.git /tmp/pip-req-build-4hqgpyxn
  Resolved https://github.com/EleutherAI/lm-evaluation-harness.git to commit d17dcea0822c542a5e75259e11cdb995f32dac44
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 15.3 MB/s eta 0:00:00
  Preparing metada

In [ ]:
!lm_eval --model hf --model_args pretrained=TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T --tasks winogrande,hellaswag,arc_challenge --device cuda:0 --num_fewshot 5 --batch_size 8 --output_path ./eval_harness/tinyllama-fp16

2024-02-05:21:22:26,361 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-02-05:21:22:26,562 INFO     [config.py:58] PyTorch version 2.1.0+cu121 available.
2024-02-05:21:22:26,563 INFO     [config.py:95] TensorFlow version 2.15.0 available.
2024-02-05:21:22:26,564 INFO     [config.py:108] JAX version 0.4.23 available.
2024-02-05 21:22:27.325150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 21:22:27.325211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 21:22:27.327210: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-05 21:22:28.521105: W ten

In [ ]:
!lm_eval --model hf --model_args pretrained=kaitchup/TinyLlama-1.1B-intermediate-step-1431k-3T-gptq-4bit --tasks winogrande,hellaswag,arc_challenge --device cuda:0 --num_fewshot 5 --batch_size 8 --output_path ./eval_harness/tinyllama-3T-gptq-4bit

2024-02-05:22:39:57,818 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-02-05:22:39:57,987 INFO     [config.py:58] PyTorch version 2.1.0+cu121 available.
2024-02-05:22:39:57,988 INFO     [config.py:95] TensorFlow version 2.15.0 available.
2024-02-05:22:39:57,989 INFO     [config.py:108] JAX version 0.4.23 available.
2024-02-05 22:39:58.518370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 22:39:58.518430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 22:39:58.519789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-05 22:39:59.686613: W ten

In [ ]:
!lm_eval --model hf --model_args pretrained=kaitchup/TinyLlama-1.1B-intermediate-step-1431k-3T-awq-4bit --tasks winogrande,hellaswag,arc_challenge --device cuda:0 --num_fewshot 5 --batch_size 8 --output_path ./eval_harness/tinyllama-3T-awq-4bit

2024-02-05:23:58:17,806 INFO     [utils.py:148] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-02-05:23:58:17,806 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-02-05:23:58:18,004 INFO     [config.py:58] PyTorch version 2.1.0+cu121 available.
2024-02-05:23:58:18,004 INFO     [config.py:95] TensorFlow version 2.15.0 available.
2024-02-05:23:58:18,005 INFO     [config.py:108] JAX version 0.4.23 available.
2024-02-05 23:58:18.734150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 23:58:18.734200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 23:58:18.735829: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to